In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [2]:
URL = 'https://storage.googleapis.com/applied-dl/heart.csv'
dataframe = pd.read_csv(URL)
dataframe.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,1,145,233,1,2,150,0,2.3,3,0,fixed,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,normal,1
2,67,1,4,120,229,0,2,129,1,2.6,2,2,reversible,0
3,37,1,3,130,250,0,0,187,0,3.5,3,0,normal,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,normal,0


In [3]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

193 train examples
49 validation examples
61 test examples


In [4]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    dataframe = dataframe.copy()
    labels = dataframe.pop('target')
    ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
#     print((dict(dataframe), labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    return ds


In [5]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [6]:
list(train_ds.as_numpy_iterator())

[({'age': array([58, 66, 68, 50, 56]),
   'sex': array([1, 1, 1, 1, 1]),
   'cp': array([3, 4, 4, 3, 3]),
   'trestbps': array([112, 120, 144, 129, 130]),
   'chol': array([230, 302, 193, 196, 256]),
   'fbs': array([0, 0, 1, 0, 1]),
   'restecg': array([2, 2, 0, 0, 2]),
   'thalach': array([165, 151, 141, 163, 142]),
   'exang': array([0, 0, 0, 0, 1]),
   'oldpeak': array([2.5, 0.4, 3.4, 0. , 0.6]),
   'slope': array([2, 2, 2, 1, 2]),
   'ca': array([1, 0, 2, 0, 1]),
   'thal': array([b'reversible', b'normal', b'reversible', b'normal', b'fixed'],
         dtype=object)},
  array([1, 0, 1, 0, 1])),
 ({'age': array([40, 53, 47, 45, 58]),
   'sex': array([1, 1, 1, 0, 1]),
   'cp': array([4, 4, 3, 2, 4]),
   'trestbps': array([110, 123, 138, 130, 125]),
   'chol': array([167, 282, 257, 234, 300]),
   'fbs': array([0, 0, 0, 0, 0]),
   'restecg': array([2, 0, 2, 2, 2]),
   'thalach': array([114,  95, 156, 175, 171]),
   'exang': array([1, 1, 0, 0, 0]),
   'oldpeak': array([2. , 2. , 0. , 0.

In [7]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['thal'])
  print('A batch of targets:', label_batch )

Every feature: ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']
A batch of ages: tf.Tensor([b'reversible' b'normal' b'normal' b'normal' b'fixed'], shape=(5,), dtype=string)
A batch of targets: tf.Tensor([1 1 0 1 1], shape=(5,), dtype=int32)


In [8]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [9]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch))

In [10]:
age = feature_column.numeric_column("age")
print(age)
demo(age)

NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[57.]
 [51.]
 [54.]
 [55.]
 [52.]], shape=(5, 1), dtype=float32)


In [11]:
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
print(age_buckets)
demo(age_buckets)

BucketizedColumn(source_column=NumericColumn(key='age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), boundaries=(18, 25, 30, 35, 40, 45, 50, 55, 60, 65))

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]], shape=(5, 11), dtype=float32)


In [26]:
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])

thal_one_hot = feature_column.indicator_column(thal)
demo(thal_one_hot)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 1. 0.]], shape=(5, 3), dtype=float32)


In [36]:
thal_hashed = feature_column.categorical_column_with_hash_bucket(
      'thal', hash_bucket_size=4)
demo(feature_column.indicator_column(thal_hashed))


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

tf.Tensor(
[[0. 0. 1. 0.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]
 [0. 0. 0. 1.]], shape=(5, 4), dtype=float32)


In [14]:
feature_columns = []

# numeric cols
for header in ['age', 'trestbps', 'chol', 'thalach', 'oldpeak', 'slope', 'ca']:
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
feature_columns.append(age_buckets)

# indicator cols
thal = feature_column.categorical_column_with_vocabulary_list(
      'thal', ['fixed', 'normal', 'reversible'])
thal_one_hot = feature_column.indicator_column(thal)
feature_columns.append(thal_one_hot)

# embedding cols
thal_embedding = feature_column.embedding_column(thal, dimension=8)
feature_columns.append(thal_embedding)

# crossed cols
crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
crossed_feature = feature_column.indicator_column(crossed_feature)
feature_columns.append(crossed_feature)

In [37]:
sex_column = feature_column.categorical_column_with_vocabulary_list(
        'sex', [1, 0])
crossed_check = feature_column.crossed_column([sex_column, thal], hash_bucket_size=6)
crossed_check = feature_column.indicator_column(crossed_check)
demo(crossed_check)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



SystemError: <built-in function TFE_Py_FastPathExecute> returned a result with an error set

In [16]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [17]:
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [21]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


model.fit(train_ds,
          validation_data=val_ds,
          epochs=5)
print(model.summary())

Train for 7 steps, validate for 2 steps
Epoch 1/5
7/7 [==============================] - 1s 159ms/step - loss: 2.8912 - accuracy: 0.5959 - val_loss: 1.9909 - val_accuracy: 0.8163
Epoch 2/5
7/7 [==============================] - 0s 16ms/step - loss: 1.5072 - accuracy: 0.6788 - val_loss: 1.3486 - val_accuracy: 0.4286
Epoch 3/5
7/7 [==============================] - 0s 16ms/step - loss: 0.9223 - accuracy: 0.6321 - val_loss: 0.6050 - val_accuracy: 0.8163
Epoch 4/5
7/7 [==============================] - 0s 14ms/step - loss: 1.0967 - accuracy: 0.7513 - val_loss: 0.3937 - val_accuracy: 0.8571
Epoch 5/5
7/7 [==============================] - 0s 15ms/step - loss: 0.7131 - accuracy: 0.7306 - val_loss: 0.5105 - val_accuracy: 0.8163
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_4 (DenseFeatu multiple                  24        
_________________________________________________

In [22]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

2/2 [==============================] - 0s 154ms/step - loss: 0.7091 - accuracy: 0.6885
Accuracy 0.6885246
